In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [2]:
URL = 'https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N'

response = requests.get(URL)

In [3]:
response.status_code

200

In [4]:
soup_TN07 = BS(response.text)

In [5]:
#print(soup_TN07.prettify())

Alright, trying to find:
- the percentage of the vote that candidate received
- the total amount raised by that candidate (as a numeric variable)
- the total amount spent by the candidate (as a numeric variable)

In [6]:
# note1: the first [0] is to get the first candidate
# note2: the second [0] is there because otherwise read_html returns a list of tables
finances = (pd
    .read_html(str
        (soup_TN07.findAll('table', attrs={'class' : 'Members--table'})[0])
    )[0]
    .pivot(index=2, columns=0, values=1)
    .reset_index()
    .drop(columns=2)
)

In [7]:
finances.head()

,Cash on Hand:,Raised:,Spent:
0,"$287,889","$1,194,960","$935,487"


In [8]:
soup_TN07.findAll('span', attrs={'class' : 'Members--vote-pct'})[0].text

'(69.9% of vote)'

In [9]:
soup_TN07.findAll('h2')[1].find('i').text

'Incumbent'

In [10]:
soup_TN07.findAll('h2')[1].find('span', attrs={'class' : 'winner'}).text

'Winner'

In [11]:
candidate_info = soup_TN07.findAll('h2')[1].find('strong').text

In [12]:
print(candidate_info)


Mark Green (R) • Incumbent
 • Winner 
(69.9% of vote)



the regex below possibly includes sr., jr. and the likes

In [13]:
re.search(r'[A-Z][a-z]+ [A-Z][a-z]+ [A-Z0-9]?[a-z]?', candidate_info)[0].strip()

'Mark Green'

In [14]:
re.findall(r'\(([R,D])\)', candidate_info)[0]

'R'

In [15]:
if re.search(r'Incumbent', candidate_info)[0] == 'Incumbent':
    print('Incumbent')
else :
    print('Not Incumbent')

Incumbent


In [16]:
if re.search(r'Winner', candidate_info)[0] == 'Winner':
    print('Winner')
else :
    print('Loser')

Winner


In [17]:
re.search(r'[A-Z][a-z]+', soup_TN07.find('h1').text)[0]

'Tennessee'

In [18]:
re.findall(r'District (\d{2})', soup_TN07.find('h1').text)[0]

'07'